In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
import os
labels=classes_train = sorted(list(filter(lambda x: os.path.isdir('/Training/' +x ), os.listdir('/Training/'))))
print(labels)

['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']


In [7]:
import splitfolders
splitfolders.ratio('/Training/', output="output", seed=1337, ratio=(.8, .2))

Copying files: 2870 files [28:57,  1.65 files/s]


In [8]:
batch_size=128
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
        '/content/output/train/', 
        target_size=(512, 512),  
        batch_size=batch_size,
        classes = labels,
        class_mode='categorical')
val_datagen = ImageDataGenerator(rescale=1/255)
val_generator = val_datagen.flow_from_directory(
        '/content/output/val', 
        target_size=(512, 512),  
        batch_size=batch_size,
        classes = classes_train,
        class_mode='categorical')

Found 2294 images belonging to 4 classes.
Found 576 images belonging to 4 classes.


In [9]:
 # Model 
image_size = 512
batch_size = 32

train_datagen = keras.preprocessing.image.ImageDataGenerator()

IMG_SHAPE = (image_size, image_size, 3)

model = tf.keras.applications.MobileNet(input_shape=IMG_SHAPE, include_top=False)
model.trainable = False

model = tf.keras.Sequential([
  model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(4, activation='sigmoid')
])

model.summary()

17235968/17225924 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 16, 16, 1024)     3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 4)                 4100      
                                                                 
Total params: 3,232,964
Trainable params: 4,100
Non-trainable params: 3,228,864
_________________________________________________________________


In [10]:
from keras.callbacks import ModelCheckpoint
model.compile(optimizer = 'rmsprop',
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

In [11]:
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", 
                               monitor = 'val_acc',
                               verbose=1, 
                               save_best_only=True)

In [12]:
history = model.fit_generator(train_generator,
                                   
                                   epochs = 20,
                                   callbacks=[checkpointer],
                                   validation_data = val_generator,
                                   validation_steps = 50)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/20
18/18 [==============================] - 93s 3s/step - loss: 1.2984 - accuracy: 0.4085 - val_loss: 1.0363 - val_accuracy: 0.5469
Epoch 2/20
18/18 [==============================] - 33s 2s/step - loss: 0.9302 - accuracy: 0.6473
Epoch 3/20
18/18 [==============================] - 33s 2s/step - loss: 0.7496 - accuracy: 0.7411
Epoch 4/20
18/18 [==============================] - 33s 2s/step - loss: 0.6355 - accuracy: 0.7868
Epoch 5/20
18/18 [==============================] - 33s 2s/step - loss: 0.5622 - accuracy: 0.8178
Epoch 6/20
18/18 [==============================] - 33s 2s/step - loss: 0.5127 - accuracy: 0.8339
Epoch 7/20
18/18 [==============================] - 33s 2s/step - loss: 0.4714 - accuracy: 0.8540
Epoch 8/20
18/18 [==============================] - 33s 2s/step - loss: 0.4323 - accuracy: 0.8614
Epoch 9/20
18/18 [==============================] - 33s 2s/step - loss: 0.4105 - accuracy: 0.8662
Epoch 10/20
18/18 [==============================] - 33s 2s/step - loss: 0.3

In [14]:
#Testing our model on unseen validation dataset
model.evaluate_generator(val_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  


[0.2976827621459961, 0.8975694179534912]

Our model has a test accuracy of 89.75%
